In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [2]:
nfl = pd.read_csv('../../data/nfl_game_by_game_raw_stats.csv')

In [3]:
# Pull out only the rushing defense related stats

rush_def = nfl.copy()
rush_def = rush_def[['season', 'week', 'team', 'opponent', 'score', 'opponent_score', 'total_rushes_allowed', 
                    'total_rush_yards_allowed', 'rushing_epa_allowed', 'rush_tds_allowed']]
rush_def = rush_def.drop_duplicates()
rush_def

,season,week,team,opponent,score,opponent_score,total_rushes_allowed,total_rush_yards_allowed,rushing_epa_allowed,rush_tds_allowed
0,2014,1,ARI,LAC,18,17,24,52.0,-8.805075,1.0
1,2014,1,ATL,NO,37,34,28,139.0,4.264254,3.0
2,2014,1,BAL,CIN,16,23,24,81.0,-4.888501,0.0
3,2014,1,BUF,CHI,23,20,18,86.0,-0.055888,0.0
4,2014,1,CAR,TB,20,14,17,102.0,1.025410,0.0
...,...,...,...,...,...,...,...,...,...,...
5535,2022,10,TB,SEA,21,16,14,39.0,-3.410833,0.0
5537,2022,10,TEN,DEN,17,10,24,66.0,-2.736545,0.0
5538,2022,10,WAS,PHI,32,21,20,94.0,4.299970,1.0
5539,2022,11,GB,TEN,17,27,29,91.0,-2.641367,1.0


In [4]:
rush_def.median()
# len(rush_def[rush_def.total_rush_yards_allowed > 105])
# len(rush_def)

<ipython-input-4-5c430c7fac3a>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  rush_def.median()


season                      2018.000000
week                           9.000000
score                         23.000000
opponent_score                23.000000
total_rushes_allowed          25.000000
total_rush_yards_allowed     106.000000
rushing_epa_allowed           -1.266159
rush_tds_allowed               1.000000
dtype: float64

In [5]:
# Removing ties 

rush_def = rush_def[rush_def.score != rush_def.opponent_score]

# Add column to determine if qb won the game or not 

# qb['qb_win'] = qb.apply(lambda x: 1 if x.score > x.opponent_score else 0, axis=1)

# Add columns to transform rushing stats

rush_def['ypc_allowed'] = rush_def.total_rush_yards_allowed / rush_def.total_rushes_allowed
rush_def['epa_allowed_per_carry'] = rush_def.rushing_epa_allowed / rush_def.total_rushes_allowed
rush_def['rush_td_allowed_per_carry'] = rush_def.rush_tds_allowed / rush_def.total_rushes_allowed

# Going to do something similar to rb value system, where instead of predicting qb win you predict whether or not 
# opposing team scores less than 23 points

rush_def['rush_def_win'] = rush_def.apply(lambda x: 1 if x.total_rush_yards_allowed < 106 else 0, axis=1)

rush_def

<ipython-input-5-668f718ba9f6>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rush_def['ypc_allowed'] = rush_def.total_rush_yards_allowed / rush_def.total_rushes_allowed
<ipython-input-5-668f718ba9f6>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rush_def['epa_allowed_per_carry'] = rush_def.rushing_epa_allowed / rush_def.total_rushes_allowed
<ipython-input-5-668f718ba9f6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

,season,week,team,opponent,score,opponent_score,total_rushes_allowed,total_rush_yards_allowed,rushing_epa_allowed,rush_tds_allowed,ypc_allowed,epa_allowed_per_carry,rush_td_allowed_per_carry,rush_def_win
0,2014,1,ARI,LAC,18,17,24,52.0,-8.805075,1.0,2.166667,-0.366878,0.041667,1
1,2014,1,ATL,NO,37,34,28,139.0,4.264254,3.0,4.964286,0.152295,0.107143,0
2,2014,1,BAL,CIN,16,23,24,81.0,-4.888501,0.0,3.375000,-0.203688,0.000000,1
3,2014,1,BUF,CHI,23,20,18,86.0,-0.055888,0.0,4.777778,-0.003105,0.000000,1
4,2014,1,CAR,TB,20,14,17,102.0,1.025410,0.0,6.000000,0.060318,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5535,2022,10,TB,SEA,21,16,14,39.0,-3.410833,0.0,2.785714,-0.243631,0.000000,1
5537,2022,10,TEN,DEN,17,10,24,66.0,-2.736545,0.0,2.750000,-0.114023,0.000000,1
5538,2022,10,WAS,PHI,32,21,20,94.0,4.299970,1.0,4.700000,0.214999,0.050000,1
5539,2022,11,GB,TEN,17,27,29,91.0,-2.641367,1.0,3.137931,-0.091082,0.034483,1


In [6]:
# Function to get relevant columns for modeling

# Note 1: Not using air yards or yac yards at the moment - since it's just predicting a win it won't be any different than 
# just the total passing yards idt. Might want a future adjustment to maybe raise/lower qb value based on air yards vs yac 
# yards pct

# Note 2: Going to use passing epa and qb rush epa seperately, rather than total qb epa

# Note 3: Using qb sacks and qb hits as predictive features. Not totally on the qb, but maybe theres some logic to 
# some qbs hold the ball too long and take too many sacks
# UPDATE: No longer using qb sacks and qb hits since they are pretty big factors in the qb value model, and since they are
# not entirely on the qb i think its not fair to add that, maybe worth adding in a separate linemen grading category

from rush_defense_value_helper_functions import get_features, get_label

In [7]:
# Test Sets of last 8 weeks of 2021 season and whatever we have from 2022 season

# test_2021 = qb[(qb.season == 2021) & (qb.week >= 9)]
# test_2021_x = get_features(test_2021)
# test_2021_y = get_label(test_2021)

# test_2022 = qb[qb.season == 2022]
# test_2022_x = get_features(test_2022)
# test_2022_y = get_label(test_2022)

test = rush_def[((rush_def.season == 2021) & (rush_def.week >= 9)) | (rush_def.season == 2022)]
test_x = get_features(test)
test_y = get_label(test)

train = rush_def[(rush_def.season < 2021) | ((rush_def.season == 2021) & (rush_def.week < 9))]
train_features = get_features(train)
train_labels = get_label(train)

rush_def.to_csv('rush_defense_value_data/rush_def.csv')
train.to_csv('rush_defense_value_data/train.csv')
test.to_csv('rush_defense_value_data/test.csv')

train_x, val_x, train_y, val_y = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

In [8]:
def train_model(model, train_x, train_y, val_x, val_y):
    clf = model.fit(train_x, train_y.to_numpy().flatten())
    
    train_preds = clf.predict(train_x)
    val_preds = clf.predict(val_x)
    
    train_acc = accuracy_score(train_y, train_preds)
    val_acc = accuracy_score(val_y, val_preds)
    
    print('Training Accuracy: {}'.format(train_acc))
    print('Validation Accuracy: {}'.format(val_acc))
    
    return clf

def test_model(clf, test_x, test_y):
    preds = clf.predict(test_x)
    probs = clf.predict_proba(test_x)[:,1]
    
    acc = accuracy_score(test_y, preds)
    
    print('Testing Accuracy: {}'.format(acc))
    
    return preds, probs

In [9]:
# Logistic Regression Model

model = LogisticRegression(max_iter=1000, random_state=71)
lr_model = train_model(model, train_x, train_y, val_x, val_y)

test_preds, test_probs = test_model(lr_model, test_x, test_y)

dump(lr_model, 'rush_defense_value_saved_models/logistic_regression.joblib')

Training Accuracy: 0.7160843563109852
Validation Accuracy: 0.7320754716981132
Testing Accuracy: 0.7307692307692307


['rush_defense_value_saved_models/logistic_regression.joblib']